In [1]:
from src.activeprobing.configs.path_config import (
    RESOURCES_NMAP_SCAN_RES_DIR_PATH,
    RESOURCES_IP_RANGES_DIR_PATH,
)
aliyun_scan_res_path = RESOURCES_NMAP_SCAN_RES_DIR_PATH / "aliyun_scan_res.jsonl"
huawei_scan_res_path = RESOURCES_NMAP_SCAN_RES_DIR_PATH / "huawei_scan_res.jsonl"
amazon_scan_res_path = RESOURCES_NMAP_SCAN_RES_DIR_PATH / "amazon_scan_res.jsonl"
tencent_cloud_scan_res_path = (
    RESOURCES_NMAP_SCAN_RES_DIR_PATH / "tencent_cloud_scan_res.jsonl"
)

aliyun_scan_res_csv_path \
    = RESOURCES_NMAP_SCAN_RES_DIR_PATH / "aliyun_scan_res.csv"
huawei_scan_res_csv_path = RESOURCES_NMAP_SCAN_RES_DIR_PATH / "huawei_scan_res.csv"
amazon_scan_res_csv_path = RESOURCES_NMAP_SCAN_RES_DIR_PATH / "amazon_scan_res.csv"
tencent_cloud_scan_res_csv_path = (
    RESOURCES_NMAP_SCAN_RES_DIR_PATH / "tencent_cloud_scan_res.csv"
)


In [2]:
import json

In [3]:
def load_scan_res(scan_res_path):
    with open(scan_res_path, "r") as f:
        scan_res = [json.loads(line) for line in f]
    return scan_res


In [4]:
huawei_scan_res = load_scan_res(huawei_scan_res_path)

In [12]:
# Host seems down
for k, v in huawei_scan_res[1].items():
    print(f"{k}: {v}")

ip: 101.44.70.168
scan_type: NORMAL
timestamp: 2024-07-11T05:21:11.289258
stdout: Starting Nmap 7.95 ( https://nmap.org ) at 2024-07-11 05:20 中国标准时间
Nmap scan report for ecs-101-44-70-168.compute.hwclouds-dns.com (101.44.70.168)
Host is up (0.076s latency).
Not shown: 43 filtered tcp ports (no-response)
PORT     STATE  SERVICE
20/tcp   closed ftp-data
21/tcp   closed ftp
22/tcp   open   ssh
80/tcp   open   http
443/tcp  open   https
3306/tcp open   mysql

Nmap done: 1 IP address (1 host up) scanned in 13.64 seconds

stderr: 
return_code: 0


In [8]:
from src.activeprobing.configs.ports_config import NORMAL_SERVICE_PORTS_LIST
NORMAL_SERVICE_PORTS_LIST    

['20',
 '21',
 '22',
 '23',
 '25',
 '53',
 '80',
 '110',
 '123',
 '143',
 '161',
 '389',
 '443',
 '445',
 '587',
 '636',
 '989',
 '990',
 '993',
 '995',
 '1433',
 '1521',
 '1830',
 '1883',
 '2049',
 '3306',
 '3389',
 '3690',
 '5353',
 '5432',
 '5433',
 '5672',
 '5900',
 '5901',
 '5902',
 '5903',
 '5904',
 '5905',
 '6379',
 '8000',
 '8080',
 '8443',
 '9000',
 '9200',
 '9418',
 '11211',
 '22222',
 '27017',
 '61613']

In [9]:
import pandas as pd
def retrieve_scan_dict_res(item) -> dict:
    stdout = item["stdout"]
    scan_res_dict = {"open": "unknown"}
    __scan_res_dict = {port: "unknown" for port in NORMAL_SERVICE_PORTS_LIST}
    scan_res_dict.update(__scan_res_dict)
    scan_res_dict_service = {f"{port}_service": "unknown" for port in NORMAL_SERVICE_PORTS_LIST}
    scan_res_dict.update(scan_res_dict_service)
    if "Host seems down" in stdout:
        scan_res_dict["open"] = False
    else:
        scan_res_dict["open"] = True
        
        # Parse the stdout to get port states and services
        lines = stdout.split('\n')
        for line in lines:
            if '/tcp' in line:
                parts = line.split()
                port = parts[0].split('/')[0]  # This is now a string
                state = parts[1]
                service = parts[2]
                
                if port in NORMAL_SERVICE_PORTS_LIST:
                    scan_res_dict[port] = state
                    scan_res_dict[f"{port}_service"] = service
    
    return scan_res_dict

def process_scan_results(scan_res):
    results = []
    for scan_r in scan_res:
        scan_res_dict = retrieve_scan_dict_res(scan_r)
        final_res_dict = {
            "ip": scan_r["ip"],
        }
        final_res_dict.update(scan_res_dict)
        results.append(final_res_dict)
    df = pd.DataFrame(results)
    return df


In [22]:
huawei_scan_res_df
huawei_scan_res_df.to_csv(huawei_scan_res_csv_path, index=False)

In [23]:
aliyun_scan_res = load_scan_res(aliyun_scan_res_path)
aliyun_scan_res_df = process_scan_results(aliyun_scan_res)
aliyun_scan_res_df.to_csv(aliyun_scan_res_csv_path, index=False)

amazon_scan_res = load_scan_res(amazon_scan_res_path)
amazon_scan_res_df = process_scan_results(amazon_scan_res)
amazon_scan_res_df.to_csv(amazon_scan_res_csv_path, index=False)


In [10]:

tencent_cloud_scan_res = load_scan_res(tencent_cloud_scan_res_path)
tencent_cloud_scan_res_df = process_scan_results(tencent_cloud_scan_res)
tencent_cloud_scan_res_df.to_csv(tencent_cloud_scan_res_csv_path, index=False)